In [ ]:
import pandas as pd
import numpy as np

df = pd.read_parquet("2024_8_19T15_18_19.parquet")

In [18]:
def rearrange(issn):
    data = list(map(lambda x: (f"{x[0][1]}", f"{x[1][1]}"),issn))
    return data

def split_issn(issn):
    e_issn = None
    p_issn = None
    for i in issn:
        if "print" in i:
            p_issn = i[1]
        elif "electronic" in i:
            e_issn = i[1]
    return p_issn, e_issn

def fix_issn(row):
    issn = row['issn']
    first_pass_issn = rearrange(issn)
    p_issn, e_issn = split_issn(first_pass_issn)
    return p_issn, e_issn
    
def separate_container_title(data):
    title = data
    if isinstance(data, np.ndarray):
        title = ", ".join(data.tolist())
    return title

In [19]:
df['separated_tokens'] = df.token_vocabulary.apply(lambda x: ", ".join(sorted(x)))
df[['print_issn', 'electronic_issn']] =df.apply(fix_issn, axis = 1, result_type='expand')
df['container_title'] = df.container_title.apply(separate_container_title)

df.drop(columns=['issn', 'total_processed_ref_len'], inplace=True)

In [20]:
df.head(5)

,DOI,token_vocabulary,token_frac_refs,cleaned_references_length,total_reference_length,work_type,author,flag,member,title,container_title,separated_tokens,print_issn,electronic_issn
0,10.1108/13552520010321497,[Drucker],0.500000,28,28,journal-article,"[hays, steven w., russ‐sellers, rebecca]",No,140,On the margins of public administration?,Journal of Management History,Drucker,1355-252X,None
1,10.1021/ic0201194,[Westerhausen],0.588235,34,34,journal-article,"[blair, stuart, izod, keith, clegg, william]",No,316,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,Westerhausen,0020-1669,1520-510X
2,10.1108/02656719410051643,[supplied],0.586207,29,29,journal-article,"[wisner, joel d., eakins, stan g.]",No,140,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,supplied,0265-671X,None
3,10.1108/13665629810209066,[Teare],0.730769,26,26,journal-article,"[teare, richard e.]",Yes,140,Developing a curriculum for organizational lea...,Journal of Workplace Learning,Teare,1366-5626,None
4,10.1108/03068299710178874,[Weber],0.757576,33,33,journal-article,"[oakley, allen]",No,140,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,Weber,0306-8293,None


In [24]:
df[(df.work_type == 'journal-article') & (df.title != '')].columns

Index(['DOI', 'token_vocabulary', 'token_frac_refs',
       'cleaned_references_length', 'total_reference_length', 'work_type',
       'author', 'flag', 'member', 'title', 'container_title',
       'separated_tokens', 'print_issn', 'electronic_issn'],
      dtype='object')

In [26]:
journal_articles_df = df[(df.work_type == 'journal-article')][['DOI', 'separated_tokens', 'token_frac_refs', 'author', 'flag', 'title', 'container_title','print_issn', 'electronic_issn','member']]

In [27]:
journal_articles_df

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,member
0,10.1108/13552520010321497,Drucker,0.500000,"[hays, steven w., russ‐sellers, rebecca]",No,On the margins of public administration?,Journal of Management History,1355-252X,None,140
1,10.1021/ic0201194,Westerhausen,0.588235,"[blair, stuart, izod, keith, clegg, william]",No,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,0020-1669,1520-510X,316
2,10.1108/02656719410051643,supplied,0.586207,"[wisner, joel d., eakins, stan g.]",No,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,0265-671X,None,140
3,10.1108/13665629810209066,Teare,0.730769,"[teare, richard e.]",Yes,Developing a curriculum for organizational lea...,Journal of Workplace Learning,1366-5626,None,140
4,10.1108/03068299710178874,Weber,0.757576,"[oakley, allen]",No,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,0306-8293,None,140
...,...,...,...,...,...,...,...,...,...,...
74669,10.2324/gomu.30.7_532,Rubber,0.500000,None,No,,NIPPON GOMU KYOKAISHI,0029-022X,1884-0442,1162
74670,10.1007/s13765-013-3182-x,Codex,0.518519,"[im, moo-hyeog]",No,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,1738-2203,2234-344X,297
74672,10.2324/gomu.30.12_934,Rubber,0.536585,None,No,,NIPPON GOMU KYOKAISHI,0029-022X,1884-0442,1162
74674,10.1254/fpj1925.34.3_285,Ebenda,0.566667,"[minami, tatsuo]",No,^|^Uuml;ber die Wirkung der verschiedenen Komb...,Nippon Yakubutsugaku Zasshi (Folia Pharmacolog...,0369-4461,2185-307X,332


In [31]:
import pandas as pd
flagged_ja = pd.read_csv('flagged_ja_articles.csv')
unflagged_ja = pd.read_csv('unflagged_journal_articles.csv')

In [142]:
processed_ja_doi = flagged_ja.DOI.to_list() + unflagged_ja.DOI.to_list()
other_journal_articles = df[(~df.DOI.isin(processed_ja_doi)) & (df.work_type == 'journal-article')].copy()
other_journal_articles

,DOI,token_vocabulary,token_frac_refs,cleaned_references_length,total_reference_length,work_type,author,flag,member,title,container_title,separated_tokens,print_issn,electronic_issn
0,10.1108/13552520010321497,[Drucker],0.500000,28,28,journal-article,"[hays, steven w., russ‐sellers, rebecca]",No,140,On the margins of public administration?,Journal of Management History,Drucker,1355-252X,None
1,10.1021/ic0201194,[Westerhausen],0.588235,34,34,journal-article,"[blair, stuart, izod, keith, clegg, william]",No,316,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,Westerhausen,0020-1669,1520-510X
2,10.1108/02656719410051643,[supplied],0.586207,29,29,journal-article,"[wisner, joel d., eakins, stan g.]",No,140,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,supplied,0265-671X,None
3,10.1108/13665629810209066,[Teare],0.730769,26,26,journal-article,"[teare, richard e.]",Yes,140,Developing a curriculum for organizational lea...,Journal of Workplace Learning,Teare,1366-5626,None
4,10.1108/03068299710178874,[Weber],0.757576,33,33,journal-article,"[oakley, allen]",No,140,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,Weber,0306-8293,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74669,10.2324/gomu.30.7_532,[Rubber],0.500000,26,43,journal-article,None,No,1162,,NIPPON GOMU KYOKAISHI,Rubber,0029-022X,1884-0442
74670,10.1007/s13765-013-3182-x,[Codex],0.518519,27,27,journal-article,"[im, moo-hyeog]",No,297,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,Codex,1738-2203,2234-344X
74672,10.2324/gomu.30.12_934,[Rubber],0.536585,41,51,journal-article,None,No,1162,,NIPPON GOMU KYOKAISHI,Rubber,0029-022X,1884-0442
74674,10.1254/fpj1925.34.3_285,[Ebenda],0.566667,30,32,journal-article,"[minami, tatsuo]",No,332,^|^Uuml;ber die Wirkung der verschiedenen Komb...,Nippon Yakubutsugaku Zasshi (Folia Pharmacolog...,Ebenda,0369-4461,2185-307X


In [143]:
'10.1108/13665629810209066' in processed_ja_doi

False

In [146]:
other_journal_articles = other_journal_articles[other_journal_articles.title != ''][['DOI', 'title', 'container_title','print_issn', 'electronic_issn','member']]

In [147]:
other_journal_articles

,DOI,title,container_title,print_issn,electronic_issn,member
0,10.1108/13552520010321497,On the margins of public administration?,Journal of Management History,1355-252X,None,140
1,10.1021/ic0201194,A Homologous Series of Alkaline Earth Phosphan...,Inorganic Chemistry,0020-1669,1520-510X,316
2,10.1108/02656719410051643,A Performance Assessment of the US Baldrige Qu...,International Journal of Quality &amp; Reliabi...,0265-671X,None,140
3,10.1108/13665629810209066,Developing a curriculum for organizational lea...,Journal of Workplace Learning,1366-5626,None,140
4,10.1108/03068299710178874,Human agents and rationality in Max Weber’s so...,International Journal of Social Economics,0306-8293,None,140
...,...,...,...,...,...,...
74661,10.2478/s11533-013-0328-3,On the number of terms in the middle of almost...,Open Mathematics,None,2391-5455,374
74662,10.1186/1744-8603-9-54,Diabetes in Argentina: cost and management of ...,Globalization and Health,None,1744-8603,297
74663,10.1186/1748-7161-8-16,The pendulum swings back to scoliosis screenin...,Scoliosis,None,1748-7161,297
74670,10.1007/s13765-013-3182-x,Review of Codex Alimentarius and comparison be...,Journal of the Korean Society for Applied Biol...,1738-2203,2234-344X,297


In [35]:
unflagged_ja.head(2)

,DOI,separated_tokens,token_frac_refs,author,flag,title,container_title,print_issn,electronic_issn,ref_pge,token_counts_by_electronic_issn,token_counts_by_print_issn,token_counts_by_container_title,member
0,10.1007/s10910-019-01078-1,Simos,0.885870,"['konguetsof, a.']",No,Algorithm for the development of families of n...,Journal of Mathematical Chemistry,0259-9791,1572-8897,1.000000,110.0,110.0,110.0,297
1,10.1007/s10910-017-0762-8,Simos,0.774011,"['konguetsof, a.']",No,A generator of families of two-step numerical ...,Journal of Mathematical Chemistry,0259-9791,1572-8897,0.994382,110.0,110.0,110.0,297


In [266]:
unflagged_ja.shape

(686, 14)

In [39]:
df1 = flagged_ja[['DOI', 'title', 'container_title', 'print_issn','electronic_issn','member']]
df2 = unflagged_ja[['DOI', 'title', 'container_title', 'print_issn','electronic_issn','member']]
analyzed_ja = pd.concat([df1,df2], ignore_index=True)
analyzed_ja

,DOI,title,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,Journal Of Biochemical Technology,0974-2328,NaN,28733


In [68]:
analyzed_ja = analyzed_ja[~analyzed_ja.DOI.isin(duplicate_title_dois)].copy()
analyzed_ja

,DOI,title,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,Journal Of Biochemical Technology,0974-2328,NaN,28733


In [244]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

def get_title_doi_info(row):
    doi = row['DOI']
    title = row['title']
    return {doi: title}

def process_title_doi_info(info):
    domain_stopwords = ['Preface', 'Introduction', 'Obituary', 'OBITUARY', 'Bibliographie',
       'In Memoriam', 'Editorial', 'In memoriam',
       'Social administration digest', 'Bibliography', 'IN MEMORIAM',
       'Obituaries', 'Book review', 'Back Matter',
       'References', 'Environmental digest', 'General Assembly',
       'Autobiography', 'OBITUARIES', 'Foreword', 'Introduction to the special issue',
       'Social Administration Digest', 'News',  'Book Review',
       'Selected bibliography', 'Présentation','Bericht über Patente',
       'Results on top physics by CMS', 'Curriculum Vitae', 'NOTES', 'Notes and Comments'
       '100 Jahre Allergie: Clemens von Pirquet – sein Allergiebegriff und das ihm zugrunde liegende Krankheitsverständnis']
    title_doi_info = info.to_list()
    combine_info = {}
    for i in title_doi_info:
        for k, v in i.items():
            str = re.sub(r"\&.*?;","", v)
            str = re.sub("SUB", "", str)
            str = re.sub("SUB", "", str)
            str = re.sub(r"<.*?>", "", str)
            if str not in domain_stopwords:
                combine_info[k] = str
    return combine_info

def get_cosine_sim(data):
    titles = list(data.values())
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(titles)
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    ci = dict(zip(list(data.keys()), titles))
    #cosine_sim_df = pd.DataFrame(cosine_sim, index=c, columns=c)
    cosine_sim_df = pd.DataFrame(cosine_sim, index=list(ci.keys()), columns=list(ci.keys()))
    
    # Display the cosine similarity matrix
    #print(cosine_sim_df)
    
    # Find the most similar pairs (excluding self-similarity)
    np.fill_diagonal(cosine_sim, 0)
    return cosine_sim_df

def get_sentences(row, ci):
    doi1 = row['row_name']
    doi2 = row['col_name']
    str1 = ci[doi1]
    str2 = ci[doi2]
    return str1, str2
    
def get_most_similar_sentences(df, ci):
    # Extract values and row, column names
    arr = df.values
    index_names = df.index
    col_names = df.columns
    
    #  Get indices where such threshold is crossed; avoid diagonal elems
    R,C = np.where(np.triu(arr,1)>=0.8)
    
    # Arrange those in columns and put out as a dataframe
    out_arr = np.column_stack((index_names[R],col_names[C],arr[R,C]))
    df_out = pd.DataFrame(out_arr,columns=['row_name','col_name','cosine_sim'])

    df_out[['str1','str2']] = df_out.apply(lambda x: get_sentences(x,ci), axis = 1, result_type='expand')
    return df_out

In [ ]:
# using HF transformer to get embeddings and determining the cosine similarity from this
# using semantic similarity
from sentence_transformers import SentenceTransformer, util
import numpy as np
def get_semantic_similarity(data):
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    titles = list(data.values())
    ci = dict(zip(list(data.keys()), titles))
    embeddings = model.encode(titles)
    similarities = model.similarity(embeddings, embeddings)
    cosine_sim = np.array(similarities)
    model_sim_df = pd.DataFrame(cosine_sim, index=list(ci.keys()), columns=list(ci.keys()))
    return cosine_sim_df

In [ ]:
    
title_doi_info = analyzed_ja.apply(get_title_doi_info, axis = 1)
combine_info = process_title_doi_info(title_doi_info)
cosine_sim_df = get_cosine_sim(combine_info)
df_out = get_most_similar_sentences(cosine_sim_df, combine_info)

In [245]:

other_journal_title_doi_info = other_journal_articles.apply(get_title_doi_info, axis = 1)
other_journal_combine_info = process_title_doi_info(other_journal_title_doi_info)
other_journal_cosine_sim_df = get_cosine_sim(other_journal_combine_info)
other_journal_df_out = get_most_similar_sentences(other_journal_cosine_sim_df, other_journal_combine_info)

In [ ]:
analyzed_ja

In [254]:
def print_title(id):
    print(other_journal_df_out.loc[id].str1, " ",other_journal_df_out.loc[id].str2) 

print_title(1)

Influence of Catalytic Poisons on the Selectivity of a Cobalt Catalyst for the Fischer–Tropsch Synthesis in Terms of the Theory of Astoichiometric Components of Catalytic Reactions   Influence of Catalytic Poisons on the Selectivity of a Cobalt Catalyst for the Fischer–Tropsch Synthesis in Terms of the Theory of Astoichiometric Components of Catalytic Reactions


In [250]:
other_journal_df_out = other_journal_df_out[(other_journal_df_out.cosine_sim >= 0.94) & (other_journal_df_out.cosine_sim <=1.0)]
other_journal_df_out.shape

(275, 5)

In [246]:
other_journal_df_out.groupby('row_name').count().sort_values('col_name', ascending=False)
#other_journal_df_out['row_name_counts'] = other_journal_df_out.groupby('row_name')['col_name'].transform('count')
#other_journal_df_out

,col_name,cosine_sim,str1,str2
row_name,,,,
10.4467/23921943rp.23.021.18369,7,7,7,7
10.2494/photopolymer.26.3,6,6,6,6
10.2903/j.efsa.2023.8111,6,6,6,6
10.4467/23921943rp.23.008.18267,6,6,6,6
10.2478/acro-2020-0005,5,5,5,5
...,...,...,...,...
10.1016/s0002-9149(03)00703-3,1,1,1,1
10.1016/s0014-5793(00)01810-x,1,1,1,1
10.1016/s0016-0032(25)91042-3,1,1,1,1


In [253]:
other_journal_df_out.sort_values('cosine_sim', ascending=False)

,row_name,col_name,cosine_sim,str1,str2
1,10.3103/s0361521923060022,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...,Influence of Catalytic Poisons on the Selectiv...
540,10.1002/cca.4156,10.1002/cca.2413,1.0,How does pyronaridine plus artesunate compare ...,How does pyronaridine plus artesunate compare ...
468,10.1038/343405a0,10.1038/341013a0,1.0,The peripatetic fossils: part 5,The peripatetic fossils: part 3
470,10.1038/343305a0,10.1038/341013a0,1.0,The peripatetic fossils: part 4,The peripatetic fossils: part 3
476,10.1016/s1353-2561(03)00078-1,10.1016/s1353-2561(03)00057-4,1.0,New Editorial Board Member,New Editorial Board Member
...,...,...,...,...,...
464,10.1016/j.amc.2003.12.049,10.1016/j.cnsns.2004.01.001,0.943004,"Compactons, solitons and periodic solutions fo...",Nonlinear variants of KdV and KP equations wit...
156,10.3103/s0884591323050045,10.15407/kfnt2023.05.024,0.941986,Physical Effects from the Kyiv Meteoroid: Part 1,Physical effects from the Kyiv meteoroid. 1
696,10.1088/1742-6596/1667/1/012045,10.1016/j.nuclphysbps.2018.03.024,0.941287,XYZ states in BESIII,XYZ states at BESIII
132,10.1016/j.mcm.2011.07.017,10.1007/s10598-010-9057-7,0.941064,Some iterative methods for general nonconvex v...,Some iterative methods for nonconvex variation...


In [ ]:
other_journal_df_out[other_journal_df_out.cosine_sim == 1.0].groupby('str1').count().sort_values('str2', ascending=False).head(50)

In [ ]:
import stopwordsiso as stopwords
english_stopwords = stopwords.stopwords("en")
a = other_journal_df_out[other_journal_df_out.cosine_sim == 1.0].groupby('str1').count().sort_values('col_name', ascending=False).head(50)
domain_stopwords = ['Preface', 'Introduction', 'Obituary', 'OBITUARY', 'Bibliographie',
       'In Memoriam', 'Editorial', 'In memoriam',
       'Social administration digest', 'Bibliography', 'IN MEMORIAM',
       'Obituaries', 'Book review', 'Back Matter',
       'References', 'Environmental digest', 'General Assembly',
       'Autobiography', 'OBITUARIES', 'Foreword',
       'Social Administration Digest', 'News',  'Book Review',
       'Selected bibliography', 
       'Results on top physics by CMS', 
       '100 Jahre Allergie: Clemens von Pirquet – sein Allergiebegriff und das ihm zugrunde liegende Krankheitsverständnis']
a.index

In [118]:
df_out['row_name_counts'] = df_out.groupby('row_name')['col_name'].transform('count')
df_out

,row_name,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...
168,10.1002/cca.4206,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [69]:

title_doi_info = analyzed_ja.apply(get_title_doi_info, axis = 1)
combine_info = process_title_doi_info(title_doi_info)

In [123]:
df_out.rename(columns={'row_name': 'DOI'}, inplace=True)
df_out

,DOI,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...
168,10.1002/cca.4206,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [256]:
other_journal_df_out.rename(columns={'row_name': 'DOI'}, inplace=True)
other_journal_df_out

,DOI,col_name,cosine_sim,str1,str2
1,10.3103/s0361521923060022,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...,Influence of Catalytic Poisons on the Selectiv...
5,10.1007/s10327-013-0483-1,10.3186/jjphytopath.79.131,1.0,Studies on diseases of tropical crops,Studies on diseases of tropical crops
8,10.1051/mmnp/20138505,10.1134/s1560354709010092,0.945301,On Global Bifurcations of Three-dimensional Di...,On global bifurcations in three-dimensional di...
9,10.1016/j.jtbi.2013.09.008,10.1177/1088767912460393,0.986945,Introduction to special issue,Introduction to the Special Issue
11,10.1007/s10511-013-9295-4,10.1007/s10511-013-9277-6,0.960046,Evolution of the magnetic fields of main-seque...,Evolution of the magnetic fields of main-seque...
...,...,...,...,...,...
906,10.1134/s2079978015020028,10.1134/s207997801501001x,1.0,Methods for the determination of glucose in bl...,Methods for the determination of glucose in bl...
907,10.1515/pbj-2015-0007,10.2478/pbj-2014-0023,1.0,"A new species of Microchilus (Goodyerinae, Orc...","A new species of Microchilus (Orchidaceae, Goo..."
909,10.1134/s1064226915020084,10.1134/s1064226914090046,0.964015,Atomic and WA-systems and R-functions applied ...,"Atomic, WA-systems, and R-functions applied in..."
912,10.4011/shikizai1937.80.174,10.4011/shikizai1937.80.267,1.0,Functionalization of Nanoparticles by Grafting...,Functionalization of Nanoparticles by Grafting...


In [121]:
analyzed_ja

,DOI,title,container_title,print_issn,electronic_issn,member
0,10.1007/s10910-010-9773-4,A family of eight-step methods with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
1,10.1007/s10910-010-9728-9,Mulitstep methods with vanished phase-lag and ...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
2,10.1007/s10910-010-9718-y,High algebraic order methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
3,10.1007/s10910-011-9862-z,A family of ten-step methods with vanished pha...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
4,10.1007/s10910-018-0986-2,New multiple stages multistep method with best...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297
...,...,...,...,...,...,...
2614,10.51847/bnyzig6kji,The Function of Maintaining Body Balance in St...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2615,10.51847/f8mofsugnz,Functional Changes in the Human Body in the Mo...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2616,10.51847/excvgchspy,Influence of Regular Sports on the Quality of ...,Journal Of Biochemical Technology,0974-2328,NaN,28733
2617,10.51847/erj8ymdkpc,Physiological Reaction of the Body of Students...,Journal Of Biochemical Technology,0974-2328,NaN,28733


In [124]:
y = pd.merge(analyzed_ja, df_out, on='DOI')
y

,DOI,title,container_title,print_issn,electronic_issn,member,col_name,cosine_sim,str1,str2,row_name_counts
0,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0654-3,0.89315,A new two stages tenth algebraic order symmetr...,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.849606,A new two stages tenth algebraic order symmetr...,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,New two stages high order symmetric six-step m...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.830857,New two stages high order symmetric six-step m...,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0301-1,0.880075,A hybrid type four-step method with vanished p...,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0227-7,0.867309,A hybrid type four-step method with vanished p...,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...,...,...,...,...,...
168,10.1002/cca.4206,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,What are the benefits and harms of neuraminida...,2


In [259]:
other_y = pd.merge(other_journal_articles, other_journal_df_out, on='DOI')
other_y.drop(columns=['str1'], inplace=True)
other_y.rename(columns={'col_name': 'DOI2', 'str2': 'DOI2_title', 'row_name_counts': 'DOI_counts'}, inplace=True)
other_y

,DOI,title,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,DOI2_title
0,10.3103/s0361521923060022,Influence of Catalytic Poisons on the Selectiv...,Solid Fuel Chemistry,0361-5219,1934-8029,1627,10.31857/s0023117723060038,1.0,Influence of Catalytic Poisons on the Selectiv...
1,10.1007/s10327-013-0483-1,Studies on diseases of tropical crops,Journal of General Plant Pathology,1345-2630,1610-739X,297,10.3186/jjphytopath.79.131,1.0,Studies on diseases of tropical crops
2,10.1051/mmnp/20138505,On Global Bifurcations of Three-dimensional Di...,Mathematical Modelling of Natural Phenomena,0973-5348,1760-6101,250,10.1134/s1560354709010092,0.945301,On global bifurcations in three-dimensional di...
3,10.1016/j.jtbi.2013.09.008,Introduction to special issue,Journal of Theoretical Biology,0022-5193,None,78,10.1177/1088767912460393,0.986945,Introduction to the Special Issue
4,10.1007/s10511-013-9295-4,Evolution of the magnetic fields of main-seque...,Astrophysics,0571-7256,1573-8191,297,10.1007/s10511-013-9277-6,0.960046,Evolution of the magnetic fields of main-seque...
...,...,...,...,...,...,...,...,...,...
270,10.1134/s2079978015020028,Methods for the determination of glucose in bl...,Review Journal of Chemistry,2079-9780,2079-9799,137,10.1134/s207997801501001x,1.0,Methods for the determination of glucose in bl...
271,10.1515/pbj-2015-0007,"A new species of Microchilus (Goodyerinae, Orc...",Polish Botanical Journal,None,2084-4352,21052,10.2478/pbj-2014-0023,1.0,"A new species of Microchilus (Orchidaceae, Goo..."
272,10.1134/s1064226915020084,Atomic and WA-systems and R-functions applied ...,Journal of Communications Technology and Elect...,1064-2269,1555-6557,137,10.1134/s1064226914090046,0.964015,"Atomic, WA-systems, and R-functions applied in..."
273,10.4011/shikizai1937.80.174,Functionalization of Nanoparticles by Grafting...,Journal of the Japan Society of Colour Material,0010-180X,None,2426,10.4011/shikizai1937.80.267,1.0,Functionalization of Nanoparticles by Grafting...


In [127]:
#y.rename(columns={'col_name': 'DOI2', 'str2': 'DOI2_title', 'row_name_counts': 'DOI_counts'}, inplace=True)
y.drop(columns=['str1'], inplace=True)
y

,DOI,title,container_title,print_issn,electronic_issn,member,DOI2,cosine_sim,DOI2_title,DOI_counts
0,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0654-3,0.89315,A family of two stages tenth algebraic order s...,2
1,10.1007/s10910-016-0674-z,A new two stages tenth algebraic order symmetr...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.849606,A new high algebraic order four stages symmetr...,2
2,10.1007/s10910-016-0691-y,New two stages high order symmetric six-step m...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-016-0627-6,0.830857,A new high algebraic order four stages symmetr...,1
3,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0301-1,0.880075,A Runge–Kutta type four-step method with vanis...,2
4,10.1007/s10910-014-0375-4,A hybrid type four-step method with vanished p...,Journal of Mathematical Chemistry,0259-9791,1572-8897,297,10.1007/s10910-013-0227-7,0.867309,A new four-step hybrid type method with vanish...,2
...,...,...,...,...,...,...,...,...,...,...
168,10.1002/cca.4206,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,2
169,10.1002/cca.3279,Can physical interventions help reduce the spr...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.2965,1.0,Can physical interventions help reduce the spr...,1
170,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.991,0.825229,What are the benefits and harms of neuraminida...,2
171,10.1002/cca.990,What are the benefits and harms of neuraminida...,Cochrane Clinical Answers,NaN,2050-4217,311,10.1002/cca.992,0.905425,What are the benefits and harms of neuraminida...,2


In [131]:
y['container_title_counts'] = y.groupby('container_title')['DOI'].transform('count')

In [260]:
other_y['container_title_counts'] = other_y.groupby('container_title')['DOI'].transform('count')

In [264]:
other_y['DOI_counts'] = other_y.groupby('DOI')['title'].transform('count')

In [133]:
y.to_csv('possible_similar_titles_flagged_unflagged_journal_titles.csv', index=False)

In [265]:
other_y.to_csv('possible_similar_titles_other_journal_titles.csv', index=False)

In [ ]:
import pandas as pd
